### Задача

Сделать класс с моделью, где происходил бы весь пайплайн обработки результатов предсказаний модели. Сделать конфигурационный файл, в котором прописать на какой девайс устанавливать модель, какой трешхолд для отбора предсказаний использовать.

При обучении модель могла предсказать одновременно `n` классов из группы лейблов `Классификация помещения` и по хорошему она должна была обучить, предсказывать только один класс. Но предсказания одновременно двух классов не исключено, поэтом при предобработке предсказаний нужно учесть, что модель может предсказать одновременно два класса из одной группы. Такая же ситуация и с лейблами `floor_fnish`, `floor_init`, `floor_without` и `wall_finish`, `wall_init`, `wall_without`. Необходимо это учесть и в случае, если модель предсказала одновременно два-три класса, выбирать тот, который имеет большую уверенность модели.

### Список классов

**Лейблы**

индекс. tag: описание.

0. `room`: комната
1. `bathroom`: ванная комната.
2. `inner_corridor`: коридор внутри квартиры.
3. `public_place`: общественное место.
4. `floor_finish`: пол чистовая отделка. 
5. `floor_init`: пол черновая отделка.
6. `ceiling_finish`: потолок чистовая отделка.
7. `wall_finish`: стена чистовая отделка.
8. `wall_init`: стена черновая отделка.
9. `door_room`: дверь межкомнатная.
10. `toilet_yes`: унитаз есть.
11. `bath_yes`: ванна есть.
12. `esocket_yes`: электрическая розетка или выключатель есть.
13. `kitchen_furniture_yes`: кухонная мебель есть.
14. `radiator`: батарея.
15. `floor_without`: пол без отделки.
16. `ceiling_without`: потолок без отделки.
17. `wall_without`: стены без отделки.
18. `window_trim`: отделка окна (подоконник, откосы).
19. `window`: проем для окна.
20. `floor_pipes`: напольные трубы отопления.
21. `garbage`: мусор.
22. `place_door`: проем для двери.
23. `place_radiator`: место для радиатора.
24. `ventilation_shaft`: вентиляционная шахта.
25. `sink_yes`: раковина есть.
26. `conditioner`: кондиционер.
27. `smoke_detector`: датчик дыма.
28. `ventilation_grille`: решетка вентиляции.
29. `fire_detectors`: извещатель (запуск сигнала о пожаре).
30. `dynamics`: динамик.
31. `switch_box_flat`: электрический щиток квартирный.
32. `switch_box_public`: электрический щиток моп.
33. `xbk`: канализационная труба.
34. `water_pipes_mop`: стояки ГВС и ХВС в МОП.
35. `water_pipes_flat`: стояки ГВС и ХВС в квартире.
36. `water_meter`: счетчик воды.
37. `heating_riser`: стояк отопления (подводка к батареи).
38. `smoke_exhaust_valve`: клапан дымоудаления.
39. `electrical_cable`: электрический кабель.
40. `electrical_riser`: разводка магистральных стояков.
41. `lamp`: светильники.
42. `door_mop`: дверь МОП.
43. `door_entrance`: дверь входная в квартиру.
44. `socket_box`: подрозетники и выключатели.
45. `tongue_groove _blocks`: пазогребневые блоки.
46. `tile_plinth`: плиточный плинтус.
47. `pantry`: кладовка.
48. `bad_light`: кадр с плохим освещением.

### Разработка

In [184]:
import json

path_anns = '../data/datasets/original/annotations/default.json'

with open(path_anns, 'r') as f:
    data = json.load(f)

list_labels = [d['name'] for d in data['categories']['label']['labels']]
print(list_labels)
print(len(list_labels))

['room', 'bathroom', 'inner_corridor', 'public_place', 'floor_finish', 'floor_init', 'ceiling_finish', 'wall_finish', 'wall_init', 'door_room', 'toilet_yes', 'bath_yes', 'esocket_yes', 'kitchen_furniture_yes', 'radiator', 'floor_without', 'ceiling_without', 'wall_without', 'window_trim', 'window', 'floor_pipes', 'garbage', 'place_door', 'place_radiator', 'ventilation_shaft', 'sink_yes', 'conditioner', 'smoke_detector', 'ventilation_grille', 'fire_detectors', 'dynamics', 'switch_box_flat', 'switch_box_public', 'xbk', 'water_pipes_mop', 'water_pipes_flat', 'water_meter', 'heating_riser', 'smoke_exhaust_valve', 'electrical_cable', 'electrical_riser', 'lamp', 'door_mop', 'door_entrance', 'socket_box', 'tongue_groove _blocks', 'tile_plinth', 'pantry', 'bad_light']
49


In [185]:
class Config:
    """Base application configs."""
    
    DEVICE = 'cpu' # 'cuda:2'
    THRESHOLD = 0.5
    BUILD_MODEL_WEIGHTS_PATH = '../logs/augoff/augoff_prec_0.737.pth'
    NUM_CLASSES = 36
    BATCH_SIZE = 32

In [186]:
import cv2
import timm
import torch
from tqdm import tqdm
from pathlib import Path

n_classes = len(list_labels)
#name_experiment = 'augoff'
device = 'cpu' # 'cuda:2'
batch_size = 32
threshold = 0.5

#path_weights = f'../logs/{name_experiment}/{name_experiment}_prec_0.761.pth'

model = timm.create_model('convnext_large_in22k', pretrained=False, num_classes=n_classes)
#checkpoint = torch.load(path_weights, map_location=torch.device(device))
#model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((192,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (norm): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=192, out_features=768, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (fc2): Linear(in_features=768, out_features=192, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (norm): LayerNorm((192,), eps=1e-06, elemen

In [187]:
import albumentations as A
import albumentations.pytorch



transforms = A.Compose(
    [
        A.Resize(width=224, height=224),
        A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
            ),
        A.pytorch.ToTensorV2()
    ]
)

In [188]:
import cv2
import torch
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader



class ImageDataset(Dataset):
    def __init__(self, path_imgs: str, transforms=None):
        """
        :param path_imgs: path to image folder.
        """

        self.path_imgs = [str(path_img) for path_img in Path(path_imgs).rglob('*.jpg')]
        self.transforms = transforms
    
    def __len__(self):
        return len(self.path_imgs)
    
    def __getitem__(self, index):
        
        path_img = self.path_imgs[index]

        img = cv2.imread(path_img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            transformered = self.transforms(image=img)
            image_tensor = transformered['image']
                    
        return image_tensor

In [198]:
[str(path_img) for path_img in Path(path_imgs).rglob('*.jpg')]

[]

In [199]:
path_imgs

'../data/test_pipeline'

In [200]:
batch_size = 32

path_imgs = '../data/datasets/test_pipeline'

ds = ImageDataset(path_imgs, transforms)
dl = DataLoader(ds, batch_size=batch_size, shuffle=False)

### Select images by bad_light

В текущей модели у нас добавлен лейбл `bad_light`, которым помечаются изображения с площим освещением. Данные снимки брать в обработку не нужно, поэтому на первом шаге, необходимо отфильтровать кадры с хорошим освещением.

Смоделируем outputs модели, где по столбцам вероятность принадлежности к классу `i`, по строкам предсказания для `j` кадра.

In [163]:
import torch

# кол-во кадров
num_rows = 3
# кол-во лейблов + 1 (функция arange не формирует последний индекс)  
num_cols = 49
threshold = 0.5

outputs = torch.rand((num_rows, num_cols))
outputs

tensor([[0.0132, 0.1191, 0.4634, 0.5433, 0.6422, 0.2907, 0.1031, 0.6845, 0.4375,
         0.9361, 0.9824, 0.7810, 0.5580, 0.3165, 0.5501, 0.3600, 0.9709, 0.4909,
         0.3307, 0.1969, 0.4762, 0.6966, 0.0160, 0.2784, 0.7320, 0.7752, 0.3759,
         0.6463, 0.0125, 0.7651, 0.1798, 0.6887, 0.2308, 0.2498, 0.8134, 0.9904,
         0.9283, 0.3224, 0.7209, 0.3008, 0.0240, 0.8241, 0.1160, 0.3642, 0.4064,
         0.4140, 0.6763, 0.0366, 0.0945],
        [0.2843, 0.8375, 0.6709, 0.4286, 0.4039, 0.2979, 0.3247, 0.0804, 0.7437,
         0.1550, 0.4121, 0.8519, 0.3496, 0.2050, 0.0592, 0.4823, 0.6536, 0.2913,
         0.2144, 0.8003, 0.6498, 0.5177, 0.5357, 0.3939, 0.9848, 0.3882, 0.8695,
         0.4248, 0.7572, 0.8036, 0.7851, 0.4504, 0.6061, 0.7080, 0.2611, 0.7825,
         0.7100, 0.9793, 0.1161, 0.4674, 0.3864, 0.0169, 0.2645, 0.3319, 0.1123,
         0.2996, 0.3205, 0.7803, 0.9383],
        [0.2246, 0.0456, 0.6384, 0.7444, 0.5565, 0.2161, 0.7019, 0.3620, 0.3872,
         0.4568, 0.2382, 

In [164]:
outputs.shape

torch.Size([3, 49])

In [165]:
outputs[:, -1]

tensor([0.0945, 0.9383, 0.1133])

In [166]:
threshold = 0.5

In [167]:
selected_outputs = outputs[outputs[:, 48] < threshold]

In [168]:
selected_outputs.shape

torch.Size([2, 49])

In [169]:
selected_outputs[:, -1]

tensor([0.0945, 0.1133])

### Regrouping

Чтобы удобно было обработать предсказания модели, нужно перегруппировать лейблы. Логически сгруппировать лейблы, чтобы в группе мы могли отобрать только одно предсказание. И все остальные, где может быть любое кол-во лейблов. 

Смоделируем outputs модели, где по столбцам вероятность принадлежности к классу `i`, по строкам предсказания для `j` кадра.

индекс лейбл описание

**Классификация помещения**

0 `room`: комната.

1 `bathroom`: ванная комната.

2 `inner_corridor`: коридор внутри квартиры.

3 `public_place`: общественное место.

47 `pantry`: кладовка.

**Отделка пола**

4 `floor_finish`: пол чистовая отделка.

5 `floor_init`: пол черновая отделка.

15 `floor_without`: пол без отделки.

**Отделка потолка**

6 `ceiling_finish`: потолок чистовая отделка.

16 `ceiling_without`: потолок без отделки.

**Отделка стен**

7 `wall_finish`: стена чистовая отделка.

8 `wall_init`: стена черновая отделка.

17 `wall_without`: стены без отделки.

**Все остальные**

9 `door_room`: дверь есть.

10 `toilet_yes`: унитаз есть.

11 `bath_yes`: ванна есть.

12 `esocket_yes`: электрическая розетка или выключатель есть.

13 `kitchen_furniture_yes`: кухонная мебель есть.

14 `radiator`: батарея.

18 `window_trim`: отделка окна (подоконник, откосы).

19 `window`: проем для окна.

20 `floor_pipes`: напольные трубы отопления.

21 `garbage`: мусор.

22 `place_door`: проем для двери.

23 `place_radiator`: место для радиатора.

24 `ventilation_shaft`: вентиляционная шахта.

25 `sink_yes`: раковина есть.

26 `conditioner`: кондиционер.

27 `smoke_detector`: датчик дыма.

28 `ventilation_grille`: решетка вентиляции.

29 `fire_detectors`: извещатель (запуск сигнала о пожаре).

30 `dynamics`: динамик.

31 `switch_box_flat`: электрический щиток квартирный.

32 `switch_box_public`: электрический щиток моп.

33 `xbk`: канализационная труба.

34 `water_pipes_mop`: стояки ГВС и ХВС в МОП.

35 `water_pipes_flat`: стояки ГВС и ХВС в квартире.

36 `water_meter`: счетчик воды.

37 `heating_riser`: стояк отопления (подводка к батареи).

38 `smoke_exhaust_valve`: клапан дымоудаления.

39 `electrical_cable`: электрический кабель.

40 `electrical_riser`: разводка магистральных стояков.

41 `lamp`: светильники.

42 `door_mop`: дверь МОП.

43 `door_entrance`: дверь входная в квартиру.

44 `socket_box`: подрозетники и выключатели.

45 `tongue_groove _blocks`: пазогребневые блоки.

46 `tile_plinth`: плиточный плинтус.

48 `bad_light`: кадр с плохим освещением.

Создадим последовательность индексов так, чтобы в каждом столбце был i-й индекс. Чтобы по индексам сверять правильность перегрупировки.

In [170]:
import torch

# кол-во кадров
num_rows = 3
# кол-во лейблов + 1 (функция arange не формирует последний индекс)  
num_cols = 49

sequence = torch.arange(num_cols)

outputs = sequence.repeat(num_rows, 1)

print(outputs)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]])


In [171]:
rooms = torch.hstack((outputs[:, :4], outputs[:, 47].unsqueeze(1)))
rooms

tensor([[ 0,  1,  2,  3, 47],
        [ 0,  1,  2,  3, 47],
        [ 0,  1,  2,  3, 47]])

In [172]:
floors = torch.hstack((outputs[:, 4:6], outputs[:, 15].unsqueeze(1)))
floors

tensor([[ 4,  5, 15],
        [ 4,  5, 15],
        [ 4,  5, 15]])

In [173]:
ceilings = torch.hstack((outputs[:, 6].unsqueeze(1), outputs[:, 16].unsqueeze(1)))
ceilings

tensor([[ 6, 16],
        [ 6, 16],
        [ 6, 16]])

In [174]:
walls = torch.hstack((outputs[:, 7:9], outputs[:, 17].unsqueeze(1)))
walls

tensor([[ 7,  8, 17],
        [ 7,  8, 17],
        [ 7,  8, 17]])

In [175]:
exclude = [15, 16, 17, 47]

column_keep = torch.tensor([x for x in range(9, 49) if x not in exclude])
column_keep

tensor([ 9, 10, 11, 12, 13, 14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
        30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48])

In [176]:
other = torch.index_select(outputs, dim=1, index=column_keep)
other

tensor([[ 9, 10, 11, 12, 13, 14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
         30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48],
        [ 9, 10, 11, 12, 13, 14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
         30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48],
        [ 9, 10, 11, 12, 13, 14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
         30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48]])

In [177]:
regrouped = torch.hstack((rooms, floors, ceilings, walls, other))
regrouped

tensor([[ 0,  1,  2,  3, 47,  4,  5, 15,  6, 16,  7,  8, 17,  9, 10, 11, 12, 13,
         14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
         35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48],
        [ 0,  1,  2,  3, 47,  4,  5, 15,  6, 16,  7,  8, 17,  9, 10, 11, 12, 13,
         14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
         35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48],
        [ 0,  1,  2,  3, 47,  4,  5, 15,  6, 16,  7,  8, 17,  9, 10, 11, 12, 13,
         14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
         35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48]])

In [178]:
def regrouping(outputs: torch.tensor) -> torch.tensor:
    """Regroup model predictions for convenient further processing."""
    
    rooms = torch.hstack((outputs[:, :4], outputs[:, 47].unsqueeze(1)))
    floors = torch.hstack((outputs[:, 4:6], outputs[:, 15].unsqueeze(1)))
    ceilings = torch.hstack((outputs[:, 6].unsqueeze(1), outputs[:, 16].unsqueeze(1)))
    walls = torch.hstack((outputs[:, 7:9], outputs[:, 17].unsqueeze(1)))
    
    exclude = [15, 16, 17, 47]
    column_keep = torch.tensor([x for x in range(9, 49) if x not in exclude])
    other = torch.index_select(outputs, dim=1, index=column_keep)
    
    return torch.hstack((rooms, floors, ceilings, walls, other))

In [179]:
regrouping(outputs)

tensor([[ 0,  1,  2,  3, 47,  4,  5, 15,  6, 16,  7,  8, 17,  9, 10, 11, 12, 13,
         14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
         35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48],
        [ 0,  1,  2,  3, 47,  4,  5, 15,  6, 16,  7,  8, 17,  9, 10, 11, 12, 13,
         14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
         35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48],
        [ 0,  1,  2,  3, 47,  4,  5, 15,  6, 16,  7,  8, 17,  9, 10, 11, 12, 13,
         14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
         35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48]])

### New indexing

индекс лейбл описание

**Классификация помещения**

0. `room`: комната.
1. `bathroom`: ванная комната.
2. `inner_corridor`: коридор внутри квартиры.
3. `public_place`: общественное место.
4. `pantry`: кладовка.

**Отделка пола**

5. `floor_finish`: пол чистовая отделка.
6. `floor_init`: пол черновая отделка.
7. `floor_without`: пол без отделки.

**Отделка потолка**

8. `ceiling_finish`: потолок чистовая отделка.
9. `ceiling_without`: потолок без отделки.

**Отделка стен**

10. `wall_finish`: стена чистовая отделка.
11. `wall_init`: стена черновая отделка.
12. `wall_without`: стены без отделки.

**Все остальные**

13. `door_room`: дверь есть.
14. `toilet_yes`: унитаз есть.
15. `bath_yes`: ванна есть.
16. `esocket_yes`: электрическая розетка или выключатель есть.
17. `kitchen_furniture_yes`: кухонная мебель есть.
18. `radiator`: батарея.
19. `window_trim`: отделка окна (подоконник, откосы).
20. `window`: проем для окна.
21. `floor_pipes`: напольные трубы отопления.
22. `garbage`: мусор.
23. `place_door`: проем для двери.
24. `place_radiator`: место для радиатора.
25. `ventilation_shaft`: вентиляционная шахта.
26. `sink_yes`: раковина есть.
27. `conditioner`: кондиционер.
28. `smoke_detector`: датчик дыма.
29. `ventilation_grille`: решетка вентиляции.
30. `fire_detectors`: извещатель (запуск сигнала о пожаре).
31. `dynamics`: динамик.
32. `switch_box_flat`: электрический щиток квартирный.
33. `switch_box_public`: электрический щиток моп.
34. `xbk`: канализационная труба.
35. `water_pipes_mop`: стояки ГВС и ХВС в МОП.
36. `water_pipes_flat`: стояки ГВС и ХВС в квартире.
37. `water_meter`: счетчик воды.
38. `heating_riser`: стояк отопления (подводка к батареи).
39. `smoke_exhaust_valve`: клапан дымоудаления.
40. `electrical_cable`: электрический кабель.
41. `electrical_riser`: разводка магистральных стояков.
42. `lamp`: светильники.
43. `door_mop`: дверь МОП.
44. `door_entrance`: дверь входная в квартиру.
45. `socket_box`: подрозетники и выключатели.
46. `tongue_groove_blocks`: пазогребневые блоки.
47. `tile_plinth`: плиточный плинтус. 
48. `bad_light`: кадр с плохим освещением.

Функция `get_prediction_by_condition` отбирает одно предсказание с макс. вероятность. Например, модель по трешхолду 0,5 предсказала что стена без отделки и стена в черновой отделке. 

In [202]:
def get_prediction_by_condition(
    outputs: torch.tensor,
    threshold: float,
    device: str,
    num_classes: int
) -> torch.tensor:
    '''One forecast from group of room labels. One forecast of fine or rough finish.
    '''
    
    preds = torch.where(outputs > threshold, outputs, torch.tensor(0).float().to(device))
    preds_max = torch.argmax(preds, axis=1)
    preds_oh = torch.nn.functional.one_hot(preds_max, num_classes=num_classes)
    mask = preds.sum(axis=1, dtype=torch.bool)
    preds_oh[~mask] = 0
    return preds_oh

In [205]:
all_preds = torch.empty(0, Config.NUM_CLASSES).to(Config.DEVICE)

model.eval()
with torch.no_grad():
    for data in tqdm(dl):
        data = data.to(Config.DEVICE)
            
        outputs = model(data)
        
        outputs = regrouping(outputs)
        outputs = torch.sigmoid(outputs)
        
        # only images with good lighting
        outputs = outputs[outputs[:, 48] < threshold]
        
        rooms = get_prediction_by_condition(
                                            outputs=outputs[:, :5],
                                            threshold=Config.THRESHOLD,
                                            device=Config.DEVICE,
                                            num_classes=5
        )
        
        floor = get_prediction_by_condition(
                                            outputs=outputs[:, 5:8],
                                            threshold=Config.THRESHOLD,
                                            device=Config.DEVICE,
                                            num_classes=3
        )
        
        ceiling = get_prediction_by_condition(
                                            outputs=outputs[:, 8:10],
                                            threshold=Config.THRESHOLD,
                                            device=Config.DEVICE,
                                            num_classes=2
        )
        
        wall = get_prediction_by_condition(
                                            outputs=outputs[:, 10:13],
                                            threshold=Config.THRESHOLD,
                                            device=Config.DEVICE,
                                            num_classes=3
        )
        
        items = torch.where(outputs[:, 13:] > threshold, 1, 0)
                
        preds = torch.hstack((rooms, floor, ceiling, wall, items))

100%|██████████| 1/1 [00:01<00:00,  1.13s/it]


In [206]:
preds.shape

torch.Size([2, 49])

In [12]:
# сделать проверку, что one hot только те вектора в которых модель сделала предсказания по трешхолду
outputs = outputs[:, :4]

# определим места, где модель сделала предсказания по трешхолду, остальные места занулим
preds = torch.where(outputs > threshold, outputs, torch.tensor(0).float().to(device))

# найдем максимальные значения в предсказании
preds_max = torch.argmax(preds, axis=1)

# сделаем one-hot вектора на основе индексов маскимальных значений
preds_oh = torch.nn.functional.one_hot(preds_max, num_classes=4)

# сделаем маску, выделим вектора со всеми нулями как false
mask = preds.sum(axis=1, dtype=torch.bool)

# по маске тем предсказаниям, в которых модель ничего не предсказала, поставим 0
preds_oh[~mask] = 0

preds_oh

tensor([[0, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 0]])

Агрегировать предсказания по помещениям.

In [20]:
id_label = {
    0: 'floor_finish',
    1: 'floor_init',
    2: 'floor_without',
    3: 'ceiling_finish',
    4: 'ceiling_without',
    5: 'wall_finish',
    6: 'wall_init',
    7: 'wall_without',
    8: 'door_room',
    9: 'toilet_yes',
    10: 'bath_yes',
    11: 'esocket_yes',
    12: 'kitchen_furniture_yes',
    13: 'radiator',
    14: 'window_trim',
    15: 'window',
    16: 'floor_pipes',
    17: 'garbage',
    18: 'place_door',
    19: 'place_radiator',
    20: 'ventilation_shaft',
    21: 'sink_yes',
    22: 'conditioner',
    23: 'smoke_detector',
    24: 'ventilation_grille',
    25: 'fire_detectors',
    26: 'dynamics',
    27: 'switch_box_flat',
    28: 'switch_box_public',
    29: 'xbk',
    30: 'water_pipes_mop',
    31: 'water_pipes_flat',
    32: 'water_meter',
    33: 'heating_riser',
    34: 'smoke_exhaust_valve',
    35: 'electrical_cable',
    36: 'electrical_riser',
    37: 'lamp',
    38: 'door_mop',
    39: 'door_entrance',
    40: 'socket_box',
    41: 'tongue_groove_blocks',
    42: 'tile_plinth',
    43: 'bad_light'
}

In [21]:
rooms = all_preds[all_preds[:, 0] == 1]
rooms = rooms.sum(axis=0).detach().cpu().numpy().astype(int)[4:]

In [22]:
bathrooms = all_preds[all_preds[:, 1] == 1]
bathrooms = bathrooms.sum(axis=0).detach().cpu().numpy().astype(int)[4:]

In [23]:
inner_coridors = all_preds[all_preds[:, 2] == 1]
inner_coridors = inner_coridors.sum(axis=0).detach().cpu().numpy().astype(int)[4:]

In [24]:
public_places = all_preds[all_preds[:, 3] == 1]
public_places = public_places.sum(axis=0).detach().cpu().numpy().astype(int)[4:]

In [ ]:
pantries = all_preds[all_preds[:, 4] == 1]
pantries = pantries.sum(axis=0).detach().cpu().numpy().astype(int)[4:]

In [25]:
preds = {
    'room': {id_label[i]:val for i, val in enumerate(rooms)},
    'bathroom': {id_label[i]:val for i, val in enumerate(bathrooms)},
    'inner_coridors': {id_label[i]:val for i, val in enumerate(inner_coridors)},
    'public_places': {id_label[i]:val for i, val in enumerate(public_places)},
    'pantries': {id_label[i]:val for i, val in enumerate(pantries)}
}

In [26]:
preds

{'room': {'floor_finish': 0,
  'floor_init': 0,
  'floor_without': 0,
  'ceiling_finish': 0,
  'ceiling_without': 0,
  'wall_finish': 0,
  'wall_init': 0,
  'wall_without': 0,
  'door_yes': 0,
  'toilet_yes': 0,
  'bath_yes': 0,
  'esocket_yes': 0,
  'kitchen_furniture_yes': 0,
  'radiator': 0,
  'window_trim': 0,
  'window': 0,
  'floor_pipes': 0,
  'garbage': 0,
  'place_door': 0,
  'place_radiator': 0,
  'place_toilet': 0,
  'place_sink': 0,
  'place_bath': 0,
  'ventilation_shaft': 0,
  'sink_yes': 0,
  'conditioner': 0,
  'smoke_detector': 0,
  'ventilation_grille': 0,
  'fire_detectors': 0,
  'dynamics': 0,
  'switch_box_flat': 0,
  'switch_box_public': 0},
 'bathroom': {'floor_finish': 1,
  'floor_init': 0,
  'floor_without': 0,
  'ceiling_finish': 1,
  'ceiling_without': 1,
  'wall_finish': 2,
  'wall_init': 0,
  'wall_without': 0,
  'door_yes': 0,
  'toilet_yes': 1,
  'bath_yes': 2,
  'esocket_yes': 2,
  'kitchen_furniture_yes': 2,
  'radiator': 0,
  'window_trim': 2,
  'windo

Отбор помещений в цикл добавим и проверим.

In [27]:
list_rooms = []

for i in range(4):
    room = all_preds[all_preds[:, i] == 1]
    room = room.sum(axis=0).detach().cpu().numpy().astype(int)[4:]
    list_rooms.append(room)

In [28]:
preds = {
    'room': {id_label[i]:val for i, val in enumerate(list_rooms[0])},
    'bathroom': {id_label[i]:val for i, val in enumerate(list_rooms[1])},
    'inner_coridors': {id_label[i]:val for i, val in enumerate(list_rooms[2])},
    'public_places': {id_label[i]:val for i, val in enumerate(list_rooms[3])},
    'pantries': {id_label[i]:val for i, val in enumerate(list_rooms[4])}
}

preds

{'room': {'floor_finish': 0,
  'floor_init': 0,
  'floor_without': 0,
  'ceiling_finish': 0,
  'ceiling_without': 0,
  'wall_finish': 0,
  'wall_init': 0,
  'wall_without': 0,
  'door_yes': 0,
  'toilet_yes': 0,
  'bath_yes': 0,
  'esocket_yes': 0,
  'kitchen_furniture_yes': 0,
  'radiator': 0,
  'window_trim': 0,
  'window': 0,
  'floor_pipes': 0,
  'garbage': 0,
  'place_door': 0,
  'place_radiator': 0,
  'place_toilet': 0,
  'place_sink': 0,
  'place_bath': 0,
  'ventilation_shaft': 0,
  'sink_yes': 0,
  'conditioner': 0,
  'smoke_detector': 0,
  'ventilation_grille': 0,
  'fire_detectors': 0,
  'dynamics': 0,
  'switch_box_flat': 0,
  'switch_box_public': 0},
 'bathroom': {'floor_finish': 1,
  'floor_init': 0,
  'floor_without': 0,
  'ceiling_finish': 1,
  'ceiling_without': 1,
  'wall_finish': 2,
  'wall_init': 0,
  'wall_without': 0,
  'door_yes': 0,
  'toilet_yes': 1,
  'bath_yes': 2,
  'esocket_yes': 2,
  'kitchen_furniture_yes': 2,
  'radiator': 0,
  'window_trim': 2,
  'windo

In [30]:
def parse_predictions(all_preds: torch.tensor) -> dict:
    
    id_label ={
        0: 'floor_finish',
        1: 'floor_init',
        2: 'floor_without',
        3: 'ceiling_finish',
        4: 'ceiling_without',
        5: 'wall_finish',
        6: 'wall_init',
        7: 'wall_without',
        8: 'door_room',
        9: 'toilet_yes',
        10: 'bath_yes',
        11: 'esocket_yes',
        12: 'kitchen_furniture_yes',
        13: 'radiator',
        14: 'window_trim',
        15: 'window',
        16: 'floor_pipes',
        17: 'garbage',
        18: 'place_door',
        19: 'place_radiator',
        20: 'ventilation_shaft',
        21: 'sink_yes',
        22: 'conditioner',
        23: 'smoke_detector',
        24: 'ventilation_grille',
        25: 'fire_detectors',
        26: 'dynamics',
        27: 'switch_box_flat',
        28: 'switch_box_public',
        29: 'xbk',
        30: 'water_pipes_mop',
        31: 'water_pipes_flat',
        32: 'water_meter',
        33: 'heating_riser',
        34: 'smoke_exhaust_valve',
        35: 'electrical_cable',
        36: 'electrical_riser',
        37: 'lamp',
        38: 'door_mop',
        39: 'door_entrance',
        40: 'socket_box',
        41: 'tongue_groove_blocks',
        42: 'tile_plinth',
        43: 'bad_light'
    }
     
    list_rooms = []

    for i in range(4):
        room = all_preds[all_preds[:, i] == 1]
        room = room.sum(axis=0).detach().cpu().numpy().astype(int)[4:]
        list_rooms.append(room)
    
    preds = {
            'room': {id_label[i]:val for i, val in enumerate(list_rooms[0])},
            'bathroom': {id_label[i]:val for i, val in enumerate(list_rooms[1])},
            'inner_coridors': {id_label[i]:val for i, val in enumerate(list_rooms[2])},
            'public_places': {id_label[i]:val for i, val in enumerate(list_rooms[3])},
            'pantries': {id_label[i]:val for i, val in enumerate(list_rooms[4])}
    }
    
    return preds

### Intersection

Необходимо расчитать количество кадров, где одновременно стоит лейбл черновой отделки стены и пазогребневые блоки. Это необходимо для расчета доли черновой отделки в условии, когда в помещении есть стены из пазогребневого блока, а пазогребневый блок не штукатурят, его необходимо засчитывать как черновая отделка.

In [236]:
# сгенерируем тензор состоящий из 0 и 1
# который будет являться предсказаниями модели для конкретного помещения
# в функции в цикле происходит селект по комнате
preds = torch.randint(0, 2, (200, 49))

11 индекс это черновая отделка стены, 46 индекс это пазогребневый блок.

Получим количество снимков, где одновременно стоит лейбл черновая отделка стены и пазогребневый блок

In [237]:
len(torch.nonzero((preds[:, 11] == 1) & (preds[:, 46] == 1)))

52

Так же необходимо посчитать количество кадров, где одновременно стоит лейбл чистовой отделки пола и плиточного плинтуса. 

Бывают ситуации, когда в МОП кладут серую плитку, из-за загрязнения не видно межплиточных швов и поэтому визуально пол выглядит как стяжка пола. Единственным патерном по которому можно сказать, что на полу установлена плитка, является наличие плиточного плинтуса.

In [238]:
len(torch.nonzero((preds[:, 5] == 1) & (preds[:, 47] == 1)))

43

### Functions

In [1]:
class Config:
    """Base application configs."""
    
    DEVICE = 'cuda:0'
    THRESHOLD = 0.5
    BUILD_MODEL_WEIGHTS_PATH = '../logs/augoff/augoff_prec_0.779.pth'
    NUM_CLASSES = 49
    BATCH_SIZE = 32

In [2]:
import cv2
import torch
import numpy as np
import albumentations as A
import albumentations.pytorch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader



class ImageDataset(Dataset):
    def __init__(self, path_imgs: str):
        """
        :param path_imgs: path to image folder.
        """

        self.path_imgs = [str(path_img) for path_img in Path(path_imgs).rglob('*.jpg')]
    
    def __len__(self):
        return len(self.path_imgs)
    
    def transform(self, image: np.array):
        transforms = A.Compose(
                                [
                                    A.Resize(width=224, height=224),
                                    A.Normalize(
                                            mean=[0.485, 0.456, 0.406],
                                            std=[0.229, 0.224, 0.225],
                                            max_pixel_value=255.0,
                                        ),
                                    A.pytorch.ToTensorV2()
            ]
        )
        
        transformered = transforms(image=image)
        image_tensor = transformered['image']
        
        return image_tensor
    
    def __getitem__(self, index):
        
        path_img = self.path_imgs[index]

        img = cv2.imread(path_img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        image_tensor = self.transform(img)
        
        return image_tensor

In [3]:
import cv2
import timm
import torch
from pathlib import Path
from torch.utils.data import DataLoader



class DecorModel:
    """Multi-labeling model for interior decoration."""
    def __init__(self, config):
        
        self.config = config
        
        self.model = timm.create_model(
                                    'convnext_large_in22k',
                                    pretrained=False,
                                    num_classes=self.config.NUM_CLASSES
        )
        
        checkpoint = torch.load(
            self.config.BUILD_MODEL_WEIGHTS_PATH,
            map_location=torch.device(self.config.DEVICE)
        )
        
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.to(self.config.DEVICE)
        
    def regrouping(self, outputs: torch.tensor) -> torch.tensor:
        """Regroup model predictions for convenient further processing."""

        rooms = torch.hstack((outputs[:, :4], outputs[:, 47].unsqueeze(1)))
        floors = torch.hstack((outputs[:, 4:6], outputs[:, 15].unsqueeze(1)))
        ceilings = torch.hstack((outputs[:, 6].unsqueeze(1), outputs[:, 16].unsqueeze(1)))
        walls = torch.hstack((outputs[:, 7:9], outputs[:, 17].unsqueeze(1)))
        
        exclude = [15, 16, 17, 47]
        column_keep = torch.tensor([x for x in range(9, 49) if x not in exclude])
        other = torch.index_select(outputs, dim=1, index=column_keep.to(self.config.DEVICE))

        return torch.hstack((rooms, floors, ceilings, walls, other))
    
    def get_prediction_by_condition(
        self,
        outputs: torch.tensor,
        threshold: float,
        device: str,
        num_classes: int
    ) -> torch.tensor:
        
        """One forecast from group of room labels. One forecast of fine or rough finish.
        """

        preds = torch.where(
            outputs > self.config.THRESHOLD,
            outputs,
            torch.tensor(0).float().to(self.config.DEVICE)
        )
        preds_max = torch.argmax(preds, axis=1)
        preds_oh = torch.nn.functional.one_hot(preds_max, num_classes=num_classes)
        mask = preds.sum(axis=1, dtype=torch.bool)
        preds_oh[~mask] = 0
        return preds_oh
            
    def predict(self, dl: DataLoader):
        
        all_preds = torch.empty(0, self.config.NUM_CLASSES).to(self.config.DEVICE)
        all_preds_video = torch.empty(0, self.config.NUM_CLASSES).to(self.config.DEVICE)
        all_probs = torch.empty(0, self.config.NUM_CLASSES).to(self.config.DEVICE)

        self.model.eval()
        with torch.no_grad():
            for data in dl:

                data = data.to(self.config.DEVICE)

                outputs = self.model(data)
                outputs = self.regrouping(outputs)
                outputs = torch.sigmoid(outputs)
                
                outputs_video = outputs.clone()
                
                # only images with good lighting
                outputs = outputs[outputs[:, -1] < self.config.THRESHOLD]
                
                rooms = self.get_prediction_by_condition(
                                                    outputs=outputs[:, :5],
                                                    threshold=self.config.THRESHOLD,
                                                    device=self.config.DEVICE,
                                                    num_classes=5
                )
                
                rooms_video = self.get_prediction_by_condition(
                                                    outputs=outputs_video[:, :5],
                                                    threshold=self.config.THRESHOLD,
                                                    device=self.config.DEVICE,
                                                    num_classes=5
                )

                floor = self.get_prediction_by_condition(
                                                    outputs=outputs[:, 5:8],
                                                    threshold=self.config.THRESHOLD,
                                                    device=self.config.DEVICE,
                                                    num_classes=3
                )
                
                floor_video = self.get_prediction_by_condition(
                                                    outputs=outputs_video[:, 5:8],
                                                    threshold=self.config.THRESHOLD,
                                                    device=self.config.DEVICE,
                                                    num_classes=3
                )

                ceiling = self.get_prediction_by_condition(
                                            outputs=outputs[:, 8:10],
                                            threshold=self.config.THRESHOLD,
                                            device=self.config.DEVICE,
                                            num_classes=2
                )
                
                ceiling_video = self.get_prediction_by_condition(
                                            outputs=outputs_video[:, 8:10],
                                            threshold=self.config.THRESHOLD,
                                            device=self.config.DEVICE,
                                            num_classes=2
                )

                wall = self.get_prediction_by_condition(
                                                    outputs=outputs[:, 10:13],
                                                    threshold=self.config.THRESHOLD,
                                                    device=self.config.DEVICE,
                                                    num_classes=3
                )
                
                wall_video = self.get_prediction_by_condition(
                                                    outputs=outputs_video[:, 10:13],
                                                    threshold=self.config.THRESHOLD,
                                                    device=self.config.DEVICE,
                                                    num_classes=3
                )

                items = torch.where(outputs[:, 13:] > self.config.THRESHOLD, 1, 0)
                items_video = torch.where(outputs_video[:, 13:] > self.config.THRESHOLD, 1, 0)

                preds = torch.hstack((rooms, floor, ceiling, wall, items))
                preds_video = torch.hstack((rooms_video, floor_video, ceiling_video, wall_video, items_video))

                all_preds = torch.vstack((all_preds, preds))
                all_preds_video = torch.vstack((all_preds_video, preds_video))
                all_probs = torch.vstack((all_probs, outputs_video))
            
            all_preds = all_preds.detach().cpu().numpy().astype(int)
            all_preds_video = all_preds_video.detach().cpu().numpy().astype(int)
            all_probs = all_probs.detach().cpu().numpy()
            
        return all_preds_video, all_preds, all_probs
        
    def parse_predictions(self, all_preds: np.array) -> dict:
    
        data_room = []
        for i in range(5):
            room = all_preds[all_preds[:, i] == 1]

            inter_wi_tgb = len(np.intersect1d(np.where(room[:, 11] == 1)[0], np.where(room[:, 46] == 1)[0]))
            inter_ff_tp = len(np.intersect1d(np.where(room[:, 5] == 1)[0], np.where(room[:, 47] == 1)[0]))
            
            condition = (room[:, 46] == 1) & (room[:, 10] != 1) & (room[:, 11] != 1) & (room[:, 12] != 1)
            only_tongue_groove_blocks = room[condition].shape[0]
            
            room = room.sum(axis=0)
            data_room.append(
                {
                    'num_img': room[i],
                    'floor_finish': room[5],
                    'floor_init': room[6],
                    'floor_without': room[7],
                    'ceiling_finish': room[8],
                    'ceiling_without': room[9],
                    'wall_finish': room[10],
                    'wall_init': room[11],
                    'wall_without': room[12],
                    'door_room': room[13],
                    'toilet_yes': room[14],
                    'bath_yes': room[15],
                    'esocket_yes': room[16],
                    'kitchen_furniture_yes': room[17],
                    'radiator': room[18],
                    'window_trim': room[19],
                    'window': room[20],
                    'floor_pipes': room[21],
                    'garbage': room[22],
                    'place_door': room[23],
                    'place_radiator': room[24],
                    'ventilation_shaft': room[25],
                    'sink_yes': room[26],
                    'conditioner': room[27],
                    'smoke_detector': room[28],
                    'ventilation_grille': room[29],
                    'fire_detectors': room[30],
                    'dynamics': room[31],
                    'switch_box_flat': room[32],
                    'switch_box_public': room[33],
                    'xbk': room[34],
                    'water_pipes_mop': room[35],
                    'water_pipes_flat': room[36],
                    'water_meter': room[37],
                    'heating_riser': room[38],
                    'smoke_exhaust_valve': room[39],
                    'electrical_cable': room[40],
                    'electrical_riser': room[41],
                    'lamp': room[42],
                    'door_mop': room[43],
                    'door_entrance': room[44],
                    'socket_box': room[45],
                    'tongue_groove_blocks': room[46],
                    'tile_plinth': room[47],
                    'bad_light': room[48],
                    'inter_wi_tgb': inter_wi_tgb,
                    'inter_ff_tp': inter_ff_tp,
                    'only_tongue_groove_blocks': only_tongue_groove_blocks
                }
            )

        data = {
            'processed_imgs': all_preds.shape[0],
            'room': data_room[0],
            'bathroom': data_room[1],
            'inner_coridors': data_room[2],
            'public_places': data_room[3],
            'pantry': data_room[4]
        }
        
        return data

In [4]:
path_imgs = '../data/datasets/test_pipeline'

ds = ImageDataset(path_imgs)
dl = DataLoader(ds, batch_size=Config.BATCH_SIZE, shuffle=False)

model = DecorModel(Config)
preds_video, preds, probs = model.predict(dl)
parsed_preds = model.parse_predictions(preds)
parsed_preds

{'processed_imgs': 4,
 'room': {'num_img': 2,
  'floor_finish': 0,
  'floor_init': 1,
  'floor_without': 1,
  'ceiling_finish': 0,
  'ceiling_without': 0,
  'wall_finish': 0,
  'wall_init': 0,
  'wall_without': 2,
  'door_room': 0,
  'toilet_yes': 0,
  'bath_yes': 0,
  'esocket_yes': 0,
  'kitchen_furniture_yes': 0,
  'radiator': 0,
  'window_trim': 0,
  'window': 1,
  'floor_pipes': 0,
  'garbage': 0,
  'place_door': 0,
  'place_radiator': 1,
  'ventilation_shaft': 0,
  'sink_yes': 0,
  'conditioner': 0,
  'smoke_detector': 0,
  'ventilation_grille': 0,
  'fire_detectors': 0,
  'dynamics': 0,
  'switch_box_flat': 0,
  'switch_box_public': 0,
  'xbk': 0,
  'water_pipes_mop': 0,
  'water_pipes_flat': 0,
  'water_meter': 0,
  'heating_riser': 0,
  'smoke_exhaust_valve': 0,
  'electrical_cable': 1,
  'electrical_riser': 0,
  'lamp': 0,
  'door_mop': 0,
  'door_entrance': 0,
  'socket_box': 0,
  'tongue_groove_blocks': 1,
  'tile_plinth': 0,
  'bad_light': 0,
  'inter_wi_tgb': 0,
  'inter_

In [26]:
preds_video.shape, preds.shape

((5, 49), (4, 49))

In [28]:
probs.shape

(4, 49)

In [27]:
preds

array([[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0]])

In [29]:
probs[probs == 0]

array([], dtype=float32)

### Test

1. Отработка алгоритма на 1 изображении.
2. Отработка случая, если на изображении модель ни разу не предсказала лейбл помещения.
3. Отработка алгоритма на 1 изображении, если на изображении модель ни разу не предсказала лейбл помещения.